In [24]:
# sheet.cell(row=user_acc_row, column=user_acc_balance, value=new_bal)
# wb.save(filename)
# import mysql.connector
import cv2
import face_recognition
import os
import datetime
from tkinter import *     # FOR GUI WIDGETS
import openpyxl as py    # FOR READING AND WRITING IN EXCEL
from tkinter import messagebox
filename = "C:\\Users\\Simrah Khan\\Desktop\\prj-atmdata.xlsx"
wb = py.load_workbook(filename)
sheet = wb['Sheet1']

eye_cap = cv2.CascadeClassifier('C:/Users/Simrah Khan/AppData/Roaming/Python/Python39/site-packages/cv2/data/haarcascade_eye_tree_eyeglasses.xml')
face_cap = cv2.CascadeClassifier("C:/Users/Simrah Khan/AppData/Roaming/Python/Python39/site-packages/cv2/data/haarcascade_frontalface_default.xml")

net = cv2.dnn.readNet("dnn_model/yolov4-tiny.weights", "dnn_model/yolov4-tiny.cfg")
model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(220,220), scale=2/255)
root = Tk()
brk, faceId_list = 0, {"1": "amaan_color_new.jpeg", "2": "suzaan.jpg", "3":"zainab.jpg", "4":"zaid.jpg"}
faceNames = {"1":"Amaan", "2":"Suzaan", "3":"Zainab", "4":"Zaid"}
width = 800
height = 600
root.title("ATM SYSTEM")
root.geometry(f'+{320}+{50}')
root.minsize(width, height)
root.maxsize(width, height)
excel_row = sheet.max_row+1
excel_column = sheet.max_column+1
accnum = StringVar()
password = StringVar()
deposit_amt = IntVar()
withdraw_amt = IntVar()
checkpin = StringVar()
newpin1 = StringVar()
newpin2 = StringVar()
transfer_accnum = StringVar()
transfer_amt_val = IntVar()
atm_row = 1
atm_column = 6
faceChances = 3
faceId = ""
def disable_lr_buttons():
    for buttons in lbuttonframe.winfo_children():
        buttons['state'] = 'disabled'
    for buttons in rbuttonframe.winfo_children():
        buttons['state'] = 'disabled'

def abled_lr_buttons():
    for buttons in lbuttonframe.winfo_children():
        buttons['state'] = 'normal'
    for buttons in rbuttonframe.winfo_children():
        buttons['state'] = 'normal'

def checkpassword():
    if str(sheet.cell(row=user_acc_row, column=user_acc_pass).value) == password.get():
        delete_windowframe()
        abled_lr_buttons()
        Label(windowframe, text='Select your transaction', bg='white', fg='black', font='Arial 21').place(x=45,y=80)
        enterb['state'] = "disabled"
    else:
        messagebox.showerror("Error", "Incorrect PIN")

def click_pass(event):
    text = event.widget.cget("text")
    password.set(password.get() + str(text))

def enterpassword():
    global  e2, num, nextx, nexty, enterb
    nextx = 0
    nexty = 0
    num = 1
    Label(windowframe, text='Enter your PIN', bg='white', fg='black', font='Arial 15').place(x=64, y=14)
    e2 = Entry(windowframe, textvariable=password, font='Arial 17', bg='seashell', show='*').place(x=55, y=60)
    enterb = Button(keypadframe, text='Enter', font='Arial 13', fg='black', bg='ivory4', command=checkpassword, width=20,height=2, pady=7)
    enterb.place(x=561, y=130)
    for i in range(0,row):
        for j in range(0, column):
            b1 = Button(keypadframe, text=num, width=19, height=2, fg='black', bg='ivory4', pady=7, font='Arial 13')
            b1.place(x=nextx, y=nexty)
            b1.bind("<Button-1>", click_pass)
            b1.bind("<Button-3>", click_pass)
            nextx = nextx + 187
            num = num + 1
        nexty = nexty + 65
        nextx = 0
    Button(keypadframe, text='', font='Arial 13', fg='black', bg='ivory4', width=19, height=2, pady=7).place(x=0, y=195)
    zb = Button(keypadframe, text='0', font='Arial 13', fg='black', bg='ivory4', width=19, height=2, pady=7)
    zb.place(x=187, y=195)
    zb.bind("<Button-1>", click_pass)

# ------------------------------------------------------------------------------------------------------
def verifyFaces():
    global faceId_list, faceId, user_acc_row, user_acc_status, faceChances
    if faceChances == 0:
        messagebox.showerror("Error", f"Account number {accnum.get()} has been disabled")
        sheet.cell(row=user_acc_row, column=user_acc_status, value="D")
        print(user_acc_row)
        print(user_acc_status)
        wb.save(filename)
        accnum.set("")
        mainwindow()
        faceChances = 3
#         return
    else:
        check_seconds = True
        check_face = False
        editSeconds = False
        limit = 15
        new_seconds = 0
        face_recorded = False
        go = True
        cap = cv2.VideoCapture(0)
        skip, img = 0, "cap_img.jpg"
        tot_blink, blinkcount = 0,0
        flag = 0
        allow = True
        phone = False
        while allow:
            ret, video_data = cap.read()
            gray_image = cv2.cvtColor(video_data, cv2.COLOR_BGR2GRAY)
            gray_image = cv2.bilateralFilter(gray_image,5,1,1)
            faces = face_cap.detectMultiScale(
                gray_image,
                scaleFactor = 1.1,
                minNeighbors = 5,
                minSize = (150,150), 
                flags=cv2.CASCADE_SCALE_IMAGE
            )
            (class_ids, score, bboxes) = model.detect(video_data)
            for i in range(35):
                if 67 in class_ids:
                    allow = False
                    phone = True
                    messagebox.showerror("Error", "Mobile was detected, shuting down")
                    root.destroy()
                    break
            if len(faces) == 1:
                if check_seconds:
                    now = datetime.datetime.now()
                    seconds = now.strftime("%S")
                    if int(seconds) > 45:
                        editSeconds = True
                        break_at_seconds = abs(int(60 - (int(seconds)+limit)))
                        break_at_seconds = str(break_at_seconds)
                        if break_at_seconds == 60:
                            break_at_seconds = 59
                    else:
                        break_at_seconds = str(int(seconds) +limit)
                    print("Face seen at second:", seconds)
                    print("Break at seconds: ", break_at_seconds)
                    check_seconds = False

                new_now = datetime.datetime.now()
                new_seconds = new_now.strftime("%S")
                if editSeconds and int(new_seconds) == int(break_at_seconds):
                    go = False
                    break
                elif str(new_seconds) == str(break_at_seconds):
                    go = False
                    messagebox.showwarning("Warning!", "Recognition took too long")
                    faceChances -= 1
                    break
                skip += 1
                cv2.putText(video_data, "Face detected, no of faces = {}".format(len(faces)), (60,100),cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 2)
                for (x,y,w,h) in faces:
                    cv2.rectangle(video_data, (x,y), (x+w, y+h), (0,255,0), 2)
                    crop_face = video_data[y:y+h, x:x+w]
                    eyes = eye_cap.detectMultiScale(
                        crop_face,
                        scaleFactor = 1.5,
                        minNeighbors = 5,
                        minSize = (10,10),
                        flags = cv2.CASCADE_SCALE_IMAGE
                    )
                    if skip >= 50:
                        if len(eyes) >= 2:
                            cv2.putText(video_data, "Eyes detected", (60,130),cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 2)
                            flag = 1
                        else:
                            cv2.putText(video_data, "No eyes detected", (60,130),cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
                            if flag == 1:
                                tot_blink += 1
                                flag = 0
                    else:
                        cv2.putText(video_data, "Stabilize yourself", (30,70),cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
                    cv2.putText(video_data, "No of Blinks : {}".format(tot_blink), (30,50),cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 2)
            else:
                skip = 0
                tot_blink = 0
                cv2.putText(video_data, "No faces detected", (60,100),cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
                check_seconds = True

            cv2.imshow("Eye blink detection", video_data)
            crop_face = video_data

            if cv2.waitKey(1) == ord('q'):
                break
            if tot_blink == 2:
                cv2.imwrite(img, crop_face)
            if tot_blink >= 3:
    #             cv2.imwrite(img, crop_face)
                check_face = True
                break
        cap.release()
        cv2.destroyAllWindows()
        if tot_blink == 3:
            print(faceId_list[faceId])
            try:
                image_1 = face_recognition.load_image_file("cap_img.jpg")
                image_1_encodings = face_recognition.face_encodings(image_1)[0]        

                image_2 = face_recognition.load_image_file(faceId_list[faceId])
                image_2_encodings = face_recognition.face_encodings(image_2)[0]

                result = face_recognition.compare_faces([image_1_encodings], image_2_encodings, tolerance=0.45)
                face_recorded = True
            except:
                print("Face was not properly recorded")
                messagebox.showerror("Error", "Face was not properly recorded")
                faceChances -= 1
                verifyFaces()

        if face_recorded:
            try:
                if not phone and check_face and face_recorded:
                    if result[0] == False:
                        print("Faces do not match")
                        faceChances -= 1
                        ans = messagebox.askyesno("Error", f"Faces do not match or tresspassing detected\nDo you want to try again?\n{faceChances} attempts left")
                        if faceChances == 0:
                            verifyFaces()
                        elif ans:
                            verifyFaces()
                        else:
                            mainwindow()
                    if result[0] == True:
                        print("Faces are matched")
                        delete_windowframe()
                        messagebox.showinfo(f"Welcome", f"Face match was success!\nWelcome {faceNames[faceId]}")
                        enterpassword()    
                else:
                    print("Some kind of spoofing was detected or recognition took too long")
                    messagebox.showerror("Error", "Some kind of spoofing was detected or recognition took too long")
                    faceChances -= 1
                    verifyFaces()
            except:
                pass

# --------------------------------------------------

def checkaccnum():
    global user_acc_row, user_acc_balance, user_acc_pass, user_acc_status, user_acc_face, faceId_list, faceId
    temp = 0
    for i in range(2, 42):
        if str(sheet.cell(row=i,column=1).value) == accnum.get():
#             delete_windowframe()
            user_acc_row = i
            user_acc_balance = 2
            user_acc_pass = 3
            temp = 1
            user_acc_status = 5
            user_acc_face = 4
            faceId = str(sheet.cell(row=i,column=user_acc_face).value)
#             print(faceId_list[faceId])
            if str(sheet.cell(row=i,column=user_acc_status).value) == "E":
#                 delete_windowframe()
                verifyFaces()
#                 enterpassword()
            else:
                messagebox.showerror("Error", "Account is disabled")
    if temp == 0:
        messagebox.showerror("Error", "Invalid Account number")

def delete_windowframe():
    for widgets in windowframe.winfo_children():
        widgets.destroy()

def click_accnum(event):
    text = event.widget.cget("text")
    accnum.set(accnum.get() + str(text))

def leave():
    root.destroy()

def balance():
    delete_windowframe()
    Label(windowframe, text="Available Balance: "+str(sheet.cell(row=user_acc_row, column=user_acc_balance).value), font='Arial 17',
          fg='black', bg='white').place(x=20, y=20)
    enterb['state'] = "disabled"
    disable_lr_buttons()

def go_to_login():
    mainwindow()

def clear_entries():
    accnum.set("")
    password.set("")
    deposit_amt.set(0)
    withdraw_amt.set(0)
    checkpin.set("")
    newpin1.set("")
    newpin2.set("")
    transfer_accnum.set("")
    transfer_amt_val.set(0)

def acc_details():
    delete_windowframe()
    Label(windowframe, text='Account Number: '+ str(sheet.cell(row=user_acc_row, column=1).value), font='Arial 17',
          fg='black', bg='white').place(x=20,y=20)
    Label(windowframe, text="Available Balance: " + str(sheet.cell(row=user_acc_row, column=user_acc_balance).value),
          font='Arial 17',
          fg='black', bg='white').place(x=20, y=70)
    enterb['state'] = "disabled"
    disable_lr_buttons()

def add100():
    deposit_amt.set(deposit_amt.get() + 100)
def add200():
    deposit_amt.set(deposit_amt.get() + 200)
def add500():
    deposit_amt.set(deposit_amt.get() + 500)
def add2000():
    deposit_amt.set(deposit_amt.get() + 2000)

def sub100():
    withdraw_amt.set(withdraw_amt.get() + 100)
def sub200():
    withdraw_amt.set(withdraw_amt.get() + 200)
def sub500():
    withdraw_amt.set(withdraw_amt.get() + 500)
def sub2000():
    withdraw_amt.set(withdraw_amt.get() + 2000)

def add_to_bal():
    if deposit_amt.get() < 100:
        messagebox.showerror("Deposit", "Minimum Denominition is 100")
    elif deposit_amt.get() > 20000:
        messagebox.showerror("Deposit", "Deposit limit Crossed")
    else:
        new_bal = int(sheet.cell(row=user_acc_row, column=user_acc_balance).value) + deposit_amt.get()
        sheet.cell(row=user_acc_row, column=user_acc_balance, value=new_bal)
        wb.save(filename)
        messagebox.showinfo("Deposited", "Deposited Successfully, Current balance: "+str(new_bal))
        deposit_amt.set(0)
        enterb['state'] = "disabled"

def sub_to_bal():
    if withdraw_amt.get() > sheet.cell(row=user_acc_row, column=user_acc_balance).value:
        messagebox.showerror("Withdraw", "Insufficient Balance")
    elif withdraw_amt.get() < 100:
        messagebox.showerror("Withdraw", "Minimum Denominition is 100")
    elif withdraw_amt.get() > 20000:
        messagebox.showerror("Withdraw", "Withdrawal limit Crossed")
    elif withdraw_amt.get() > int(sheet.cell(row=atm_row, column=atm_column).value):
        messagebox.showerror("Withdraw", "Insufficient ATM balance")
    else:
        new_bal = int(sheet.cell(row=user_acc_row, column=user_acc_balance).value) - withdraw_amt.get()
        atm_cash = sheet.cell(row=atm_row, column=atm_column).value
        sheet.cell(row=atm_row, column=atm_column, value=(int(atm_cash)-withdraw_amt.get())) #atm cash
        sheet.cell(row=user_acc_row, column=user_acc_balance, value=new_bal)  #user balance
        wb.save(filename)
        messagebox.showinfo("Withdraw", "Withdrawn Successfully, Current balance: "+str(new_bal))
        withdraw_amt.set(0)
        enterb['state'] = "disabled"

def deposit():
    delete_windowframe()
    disable_lr_buttons()
    global enterb
    Label(windowframe, text='Select the Denomination', fg='black', bg='white', font='Arial 18').place(x=55, y=20)
    e3 = Entry(windowframe, textvariable=deposit_amt, font='Arial 17', bg='seashell')
    e3.place(x=55, y=60)
    enterb = Button(keypadframe, text='Enter', font='Arial 13', fg='black', bg='ivory4', command=add_to_bal, width=20,height=2, pady=7)
    enterb.place(x=561, y=130)
    Button(windowframe, text='+100', bg="seashell",font='Arial 12',command=add100, width=7, height=2).place(x=60, y=120)
    Button(windowframe, text='+200', bg="seashell", font='Arial 12', command=add200, width=7, height=2).place(x=240,y=120)
    Button(windowframe, text='+500', bg="seashell", font='Arial 12', command=add500, width=7, height=2).place(x=60,y=190)
    Button(windowframe, text='+2000', bg="seashell", font='Arial 12', command=add2000, width=7, height=2).place(x=240,y=190)
# WITHDRAW---------------------------------------------------------------------------------------------------------
def withdrawal():
    delete_windowframe()
    disable_lr_buttons()
    global enterb
    Label(windowframe, text='Select the Denomination', fg='black', bg='white', font='Arial 18').place(x=55, y=20)
    e3 = Entry(windowframe, textvariable=withdraw_amt, font='Arial 17', bg='seashell')
    e3.place(x=55, y=60)
    enterb = Button(keypadframe, text='Enter', font='Arial 13', fg='black', bg='ivory4', command=sub_to_bal, width=20,height=2, pady=7)
    enterb.place(x=561, y=130)
    Button(windowframe, text='+100', bg="seashell", font='Arial 12', command=sub100, width=7, height=2).place(x=60,y=120)
    Button(windowframe, text='+200', bg="seashell", font='Arial 12', command=sub200, width=7, height=2).place(x=240,y=120)
    Button(windowframe, text='+500', bg="seashell", font='Arial 12', command=sub500, width=7, height=2).place(x=60,y=190)
    Button(windowframe, text='+2000', bg="seashell", font='Arial 12', command=sub2000, width=7, height=2).place(x=240,y=190)

# CHANGE PIN CODE----------------------------------------------------------------------------------------------------------------
def click_changepin(event):
    text = event.widget.cget("text")
    checkpin.set(checkpin.get() + str(text))

def change_newpin1(event):
    text = event.widget.cget("text")
    newpin1.set(newpin1.get() + str(text))

def change_newpin2(event):
    text = event.widget.cget("text")
    newpin2.set(newpin2.get() + str(text))

def addnewpin():
    if newpin1.get() == newpin2.get():
        sheet.cell(row=user_acc_row, column=user_acc_pass, value=newpin1.get())
        wb.save(filename)
        messagebox.showinfo("Change PIN", "PIN Changed Successfully")
        #mainwindow()
        enterb['state'] = "disabled"
    else:
        messagebox.showerror("Change PIN", "PINs do not match")

def changing_pin():
    delete_windowframe()
    global enterb
    Label(windowframe, text='Enter new PIN', bg='white', fg='black', font='Arial 15').place(x=64, y=14)
    Entry(windowframe, textvariable=newpin1, font='Arial 17', bg='seashell', show='*').place(x=55, y=60)
    Label(windowframe, text='Enter new PIN again', bg='white', fg='black', font='Arial 15').place(x=64, y=110)
    Entry(windowframe, textvariable=newpin2, font='Arial 17', bg='seashell', show='*').place(x=55, y=155)
    enterb = Button(keypadframe, text='Enter', font='Arial 13', fg='black', bg='ivory4', command=addnewpin,width=20, height=2, pady=7)
    enterb.place(x=561, y=130)
    nextx = 0
    nexty = 0
    num = 1
    for i in range(0, row):
        for j in range(0, column):
            b1 = Button(keypadframe, text=num, width=19, height=2, fg='black', bg='ivory4', pady=7, font='Arial 13')
            b1.place(x=nextx, y=nexty)
            b1.bind("<Button-1>", change_newpin1)
            b1.bind("<Button-3>", change_newpin2)
            nextx = nextx + 187
            num = num + 1
        nexty = nexty + 65
        nextx = 0
    zb = Button(keypadframe, text='0', font='Arial 13', fg='black', bg='ivory4', width=19, height=2, pady=7)
    zb.place(x=187, y=195)
    zb.bind("<Button-1>", change_newpin1)
    zb.bind("<Button-3>", change_newpin2)

def check_pin():
    if checkpin.get() == str(sheet.cell(row=user_acc_row, column=user_acc_pass).value):
        changing_pin()
    else:
        messagebox.showerror("Change PIN", "Incorrect PIN")

def changepin():
    global nextx, nexty, num, enterb
    nextx = 0
    nexty = 0
    num = 1
    delete_windowframe()
    disable_lr_buttons()
    enterb = Button(keypadframe, text='Enter', font='Arial 13', fg='black', bg='ivory4',command=check_pin, width=20,height=2, pady=7)
    enterb.place(x=561, y=130)
    Label(windowframe, text='Enter old PIN', fg='black', bg='white', font='Arial 18').place(x=60, y=20)
    e3 = Entry(windowframe, textvariable=checkpin, font='Arial 17', bg='seashell', show='*')
    e3.place(x=55, y=60)
    for i in range(0,row):
        for j in range(0, column):
            b1 = Button(keypadframe, text=num, width=19, height=2, fg='black', bg='ivory4', pady=7, font='Arial 13')
            b1.place(x=nextx, y=nexty)
            b1.bind("<Button-1>", click_changepin)
            #b1.bind("<Button-3>", click_pass)
            nextx = nextx + 187
            num = num + 1
        nexty = nexty + 65
        nextx = 0
    zb = Button(keypadframe, text='0', font='Arial 13', fg='black', bg='ivory4', width=19, height=2, pady=7)
    zb.place(x=187, y=195)
    zb.bind("<Button-1>", click_changepin)
# TRANSFER CODEE -------------------------------------------------------------------------------------------------------
def click_transfer(event):
    text = event.widget.cget("text")
    transfer_accnum.set(transfer_accnum.get() + str(text))

def transfer_add100():
    transfer_amt_val.set(transfer_amt_val.get() + 100)
def transfer_add200():
    transfer_amt_val.set(transfer_amt_val.get() + 200)
def transfer_add500():
    transfer_amt_val.set(transfer_amt_val.get() + 500)
def transfer_add2000():
    transfer_amt_val.set(transfer_amt_val.get() + 2000)

def do_transfer():
    if transfer_amt_val.get() > sheet.cell(row=user_acc_row, column=user_acc_balance).value:
        messagebox.showerror("Transfer", "Transfer amount exceeds your balance")
    elif transfer_amt_val.get() < 100:
        messagebox.showerror("Transfer", "Transfer range must be 100-20000")
    elif transfer_amt_val.get() > 20000:
        messagebox.showerror("Transfer", "Transfer range must be 100-20000")
    else:
        user_new_balance = int(sheet.cell(row=user_acc_row, column=user_acc_balance).value) - transfer_amt_val.get()
        transfer_new_balance = int(sheet.cell(row=transfer_acc_row, column=transfer_acc_bal).value) + transfer_amt_val.get()
        sheet.cell(row=user_acc_row, column=user_acc_balance, value=user_new_balance)
        sheet.cell(row=transfer_acc_row, column=transfer_acc_bal, value=transfer_new_balance)
        wb.save(filename)
        messagebox.showinfo("Trnasfer", "Transfer Successful! Current Balance: " + str(user_new_balance))

        enterb['state'] = "disabled"

def transfer_amt():
    delete_windowframe()
    global enterb
    Label(windowframe, text='Select the Denomination', fg='black', bg='white', font='Arial 18').place(x=55, y=20)
    e3 = Entry(windowframe, textvariable=transfer_amt_val, font='Arial 17', bg='seashell')
    e3.place(x=55, y=60)
    enterb = Button(keypadframe, text='Enter', font='Arial 13', fg='black',command=do_transfer, bg='ivory4', width=20,height=2, pady=7)
    enterb.place(x=561, y=130)
    Button(windowframe, text='+100', bg="seashell", font='Arial 12', command=transfer_add100, width=7, height=2).place(x=60,y=120)
    Button(windowframe, text='+200', bg="seashell", font='Arial 12', command=transfer_add200, width=7, height=2).place(x=240,y=120)
    Button(windowframe, text='+500', bg="seashell", font='Arial 12', command=transfer_add500, width=7, height=2).place(x=60,y=190)
    Button(windowframe, text='+2000', bg="seashell", font='Arial 12', command=transfer_add2000, width=7, height=2).place(x=240,y=190)

def check_transfer():
    global user_acc_row, user_acc_balance, user_acc_pass
    global transfer_acc_row, transfer_acc_bal
    temp = 0
    for i in range(2, 42):
        if transfer_accnum.get() == str(sheet.cell(row=user_acc_row, column=1).value):
            temp = 2
        elif str(sheet.cell(row=i, column=1).value) == transfer_accnum.get():
            delete_windowframe()
            transfer_acc_row = i
            transfer_acc_bal = 2
            temp = 1
            #print("acc num found")
            #print(transfer_acc_row, transfer_acc_bal)
            transfer_amt()
    if temp == 0:
        messagebox.showerror("Transfer", "Couldn't find account number "+transfer_accnum.get())
    if temp == 2:
        messagebox.showerror("Transfer", "Cannot be your own account number")

def transfer():
    delete_windowframe()
    disable_lr_buttons()
    global enterb
    enterb = Button(keypadframe, text='Enter', font='Arial 13', fg='black', bg='ivory4',command=check_transfer, width=20,height=2, pady=7)
    enterb.place(x=561, y=130)
    Label(windowframe, text='Enter Account number', fg='black', bg='white', font='Arial 18').place(x=55, y=20)
    e3 = Entry(windowframe, textvariable=transfer_accnum, font='Arial 17', bg='seashell')
    e3.place(x=55, y=60)
    nextx = 0
    nexty = 0
    num = 1
    for i in range(0,row):
        for j in range(0, column):
            b1 = Button(keypadframe, text=num, width=19, height=2, fg='black', bg='ivory4', pady=7, font='Arial 13')
            b1.place(x=nextx, y=nexty)
            b1.bind("<Button-1>", click_transfer)
            nextx = nextx + 187
            num = num + 1
        nexty = nexty + 65
        nextx = 0
    zb = Button(keypadframe, text='0', font='Arial 13', fg='black', bg='ivory4', width=19, height=2, pady=7)
    zb.place(x=187, y=195)
    zb.bind("<Button-1>", click_transfer)

# MAIN PARENT FUNCTION RESPONSIBLE FOR CALLING OF OTHER FUNCTIONS---------------------------------------------------------------------------
def mainwindow():
    clear_entries()
    global accnum, password, e1, b1, windowframe, keypadframe, lbuttonframe, rbuttonframe, nextx, nexty, num, row, column
    parentframe = Frame(root, bg="ivory4", borderwidth=14, relief=RIDGE)
    parentframe.place(x=0, y=0, width=width, height=height)
    windowframe = Frame(parentframe, bg='white', border=10, relief=RIDGE)
    windowframe.place(x=186, y=0, width=400, height=height/2)
    # LEFT COLUMN BUTTONS
    lbuttonframe = Frame(parentframe, bg='snow4', borderwidth=5, relief=SUNKEN)
    lbuttonframe.place(x=5,y=0, width=174, height=(height/2))
    Button(lbuttonframe, text='Deposit', font='Arial 15 ', fg='black', bg='ivory4', width=14, height=3,command=deposit, state="disabled").place(x=0,y=0)
    Button(lbuttonframe, text='Withdraw', font='Arial 15 ', fg='black', bg='ivory4', width=14, height=3,command=withdrawal, state="disabled").place(x=0, y=100)
    Button(lbuttonframe, text='Balance', font='Arial 15 ', fg='black', bg='ivory4', width=14, height=3, command=balance, state="disabled").place(x=0, y=200)
    # RIGHT COLUMN BUTTONS
    rbuttonframe = Frame(parentframe, bg='snow4', borderwidth=5, relief=SUNKEN)
    rbuttonframe.place(x=590, y=0, width=174, height=(height / 2))
    Button(rbuttonframe, text='Change PIN', font='Arial 15 ', fg='black', bg='ivory4', width=14,command=changepin, height=3, state="disabled").place(x=0, y=0)
    Button(rbuttonframe, text='Transfer', font='Arial 15 ', fg='black', bg='ivory4', width=14, height=3,command=transfer, state="disabled").place(x=0,y=100)
    Button(rbuttonframe, text='Account details', font='Arial 15 ', fg='black', bg='ivory4', width=14, height=3,command=acc_details, state="disabled").place(x=0,y=200)
    # FRAME FOR KEYPAD
    keypadframe = Frame(parentframe, bg='snow4', borderwidth=5, relief=SUNKEN)
    keypadframe.place(x=5, y=300, width=759, height=270)
    # KEYPAD BUTTONS
    row = 3
    column = 3
    nextx = 0
    nexty = 0
    num = 1
    for i in range(0,row):
        for j in range(0, column):
            b1 = Button(keypadframe, text=num, width=19, height=2, fg='black', bg='ivory4', pady=7, font='Arial 13')
            b1.place(x=nextx, y=nexty)
            b1.bind("<Button-1>", click_accnum)
            #b1.bind("<Button-3>", click_pass)
            nextx = nextx + 187
            num = num + 1
        nexty = nexty + 65
        nextx = 0
    Button(keypadframe, text='', font='Arial 13', fg='black', bg='ivory4', width=19, height=2, pady=7).place(x=0, y=195)
    zb = Button(keypadframe, text='0', font='Arial 13', fg='black', bg='ivory4', width=19, height=2, pady=7)
    zb.place(x=187, y=195)
    zb.bind("<Button-1>", click_accnum)
    #zb.bind("<Button-3>", click_pass)
    Button(keypadframe, text='', font='Arial 13', fg='black', bg='ivory4', width=19, height=2, pady=7).place(x=374, y=195)

    # KEYPAD ENTRY BUTTONS-----------------------------------------------------------------------------------------------------------------
    cb = Button(keypadframe, text='Cancel', font='Arial 13', fg='black', bg='ivory4',command=go_to_login, width=20, height=2, pady=7)
    cb.place(x=561, y=0)
    clb = Button(keypadframe, text='Clear', font='Arial 13', fg='black', bg='ivory4',command=clear_entries, width=20, height=2, pady=7)
    clb.place(x=561, y=65)
    enterb = Button(keypadframe, text='Enter', font='Arial 13', fg='black', bg='ivory4', command=checkaccnum, width=20, height=2, pady=7)
    enterb.place(x=561, y=130)
    exitb = Button(keypadframe, text='Exit', font='Arial 13', fg='black', bg='ivory4',command=leave, width=20, height=2, pady=7)
    exitb.place(x=561, y=195)

    # LOGIN PAGE-----------------------------------------------------------------------------------------------------------------
    Label(windowframe, text='Enter your account number', bg='white', fg='black', font='Arial 15').place(x=64, y=14)
    accnum.set("")
    password.set("")
    e1 = Entry(windowframe, textvariable=accnum, font='Arial 17', bg='seashell').place(x=55, y=60)

mainwindow()
root.mainloop()
